https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download

In [1]:
from sklearn.model_selection import train_test_split
import os
import torch
import sys
from pathlib import Path

In [2]:
sys.path.append(os.path.abspath("..")) 

In [4]:
import pandas as pd
from src.textclf_transformer import * 

c:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
c:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\.venv\Lib\site-pac

In [5]:
df = pd.read_csv('../data/raw/IMDB.csv')

In [9]:
df.groupby('sentiment').count()

,review
sentiment,
negative,25000
positive,25000


In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [31]:
df["review"] = df["review"].str.replace(r"<br\s*/?>", "", regex=True)

In [32]:
# zamien positice na 1, negative na 0
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [33]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['sentiment'])

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['sentiment'])

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 40500 | Val: 4500 | Test: 5000


In [34]:
tok = WordPieceTokenizerWrapper()

In [35]:
tokens_train = tok.encode_pandas(train_df, 'review',
                                  max_length=1000, 
                                  label_col='sentiment', 
                                  tokenizer_dir="../src/textclf_transformer/tokenizer/BERT_original")
tokens_val = tok.encode_pandas(val_df, 'review',
                                max_length=1000,
                                label_col='sentiment',
                                tokenizer_dir="../src/textclf_transformer/tokenizer/BERT_original")
tokens_test = tok.encode_pandas(test_df, 'review',
                                 max_length=1000,
                                 label_col='sentiment',
                                 tokenizer_dir="../src/textclf_transformer/tokenizer/BERT_original")


In [41]:
save_dir = Path("../data/tokenized/IMDB")
save_dir.mkdir(parents=True, exist_ok=True)

torch.save(tokens_train, save_dir / "train_dataset.pt")
torch.save(tokens_val, save_dir / "val_dataset.pt")
torch.save(tokens_test, save_dir / "test_dataset.pt")